In [19]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [45]:
pip install openai faiss-cpu langchain


Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 5.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [35]:
pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.7/906.7 kB 11.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [55]:
os.environ['OPENAI_API_KEY']=''

In [57]:
llm = OpenAI(temperature = 0.9,max_tokens = 500)

loaders = UnstructuredURLLoader(urls=[
       "https://www.moneycontrol.com/news/world/donald-trump-shooting-a-look-back-at-us-presidential-assassination-attempts-12768598.html",
       "https://www.moneycontrol.com/news/trends/massively-reduced-ordering-from-swiggy-zomato-says-capitalmind-ceo-platform-charge-is-now-rs-6-12768717.html"
])
data = loaders.load()
len(data)

2

In [58]:
text_splitter  = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)
docs = text_splitter.split_documents(data)
len(docs)

47

In [ ]:

embeddings = OpenAIEmbeddings()

vectorindex_openai = FAISS.from_documents(docs, embeddings)


In [ ]:
file_path = "vector_index.pkl"
with open(file_path,"wb") as f:
    pickle.dump(vectorindex_openai,f)

In [ ]:
if os.path.exists(file_path):
    with open(file_path,"rb")as f:
        vectorIndex = pickle.load(f)

In [ ]:
chain = RetrievalQAWithSourcesChain.from_llm(llm = llm,retriever = vectorIndex.as_retriever())

In [ ]:
query = "what is the pirce of Tiago iCNG?"
langchain.debug = True
chain({"question":query},return_only_outputs = True)